In [1]:
import time
import random
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def KaliMatriks(A, B, n):
    """
    Brute-force matrix multiplication.
    Kembalikan (C, op_count) di mana op_count = # (mul + add).
    """
    C = [[0]*n for _ in range(n)]
    ops = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i][j] += A[i][k] * B[k][j]
                ops += 2    # satu multiplikasi + satu penjumlahan
    return C, ops

In [3]:
def add_matrix(A, B, bucket):
    n = len(A)
    bucket[0] += n*n                  # hitung penjumlahan
    return [[A[i][j] + B[i][j] for j in range(n)] for i in range(n)]

def sub_matrix(A, B, bucket):
    n = len(A)
    bucket[0] += n*n                  # hitung pengurangan
    return [[A[i][j] - B[i][j] for j in range(n)] for i in range(n)]
#———————————————————————————————————————
# 3. Strassen + bucket global utk ops
#———————————————————————————————————————
def KaliMatriksStrassen(A, B, n):
    bucket = [0]                      # wadah bersama
    C = _strassen_rec(A, B, n, bucket)
    return C, bucket[0]               # jaga antarmuka lama

def _strassen_rec(A, B, n, bucket):
    if n == 1:                        # basis rekursi
        bucket[0] += 1                # 1 perkalian skalar
        return [[A[0][0] * B[0][0]]]

    mid = n // 2
    # — split —
    A11 = [row[:mid] for row in A[:mid]]
    A12 = [row[mid:] for row in A[:mid]]
    A21 = [row[:mid] for row in A[mid:]]
    A22 = [row[mid:] for row in A[mid:]]

    B11 = [row[:mid] for row in B[:mid]]
    B12 = [row[mid:] for row in B[:mid]]
    B21 = [row[:mid] for row in B[mid:]]
    B22 = [row[mid:] for row in B[mid:]]

    # — 7 proyek Strassen —
    M1 = _strassen_rec(sub_matrix(A12, A22, bucket),
                       add_matrix(B21, B22, bucket), mid, bucket)
    M2 = _strassen_rec(add_matrix(A11, A22, bucket),
                       add_matrix(B11, B22, bucket), mid, bucket)
    M3 = _strassen_rec(sub_matrix(A11, A21, bucket),
                       add_matrix(B11, B12, bucket), mid, bucket)
    M4 = _strassen_rec(add_matrix(A11, A12, bucket),
                       B22, mid, bucket)
    M5 = _strassen_rec(A11,
                       sub_matrix(B12, B22, bucket), mid, bucket)
    M6 = _strassen_rec(A22,
                       sub_matrix(B21, B11, bucket), mid, bucket)
    M7 = _strassen_rec(add_matrix(A21, A22, bucket),
                       B11, mid, bucket)

    # — rakit C —
    C11 = add_matrix(sub_matrix(add_matrix(M1, M2, bucket),
                                M4, bucket), M6, bucket)
    C12 = add_matrix(M4, M5, bucket)
    C21 = add_matrix(M6, M7, bucket)
    C22 = sub_matrix(add_matrix(sub_matrix(M2, M3, bucket),
                                M5, bucket), M7, bucket)

    C = [[0]*n for _ in range(n)]
    for i in range(mid):
        C[i][:mid]       = C11[i]
        C[i][mid:]       = C12[i]
        C[mid+i][:mid]   = C21[i]
        C[mid+i][mid:]   = C22[i]
    return C

In [4]:
n_values = [2, 4, 8, 16, 32, 64, 512, 1024]
records = []

for n in n_values:
    A = [[random.randint(0,10) for _ in range(n)] for _ in range(n)]
    B = [[random.randint(0,10) for _ in range(n)] for _ in range(n)]

    t0 = time.perf_counter()
    _, bf_ops = KaliMatriks(A, B, n)
    t_bf = time.perf_counter() - t0

    t0 = time.perf_counter()
    _, st_ops = KaliMatriksStrassen(A, B, n)
    t_st = time.perf_counter() - t0

    records.append({'n': n, 'time_bf': t_bf, 'ops_bf': bf_ops,
                            'time_st': t_st, 'ops_st': st_ops})

df = pd.DataFrame(records)
print(df.to_string(index=False))

plt.figure()
plt.plot(df['n'], df['time_bf'], label='Brute-Force')
plt.plot(df['n'], df['time_st'], label='Strassen (bucket)')
plt.xlabel('n'); plt.ylabel('Waktu (s)'); plt.legend(); plt.title('Waktu vs n')

plt.figure()
plt.plot(df['n'], df['ops_bf'], label='Brute-Force')
plt.plot(df['n'], df['ops_st'], label='Strassen (bucket)')
plt.xlabel('n'); plt.ylabel('Jumlah Operasi'); plt.legend(); plt.title('Operasi vs n')
plt.show()

KeyboardInterrupt: 